# Sentinel 2 cloud free best pixel WF

remove cloud and shadow from series of S2 ARD images
then mosaic them to create a best pixel image for given time period

# steps:

Run a pyeodh stac search for given time period


In [2]:
import pyeodh
import xarray as xr

# pyeodh.set_log_level(10)
rc = pyeodh.Client().get_catalog_service()

thetford_pnt = {
    "coordinates": [
        [
            [0.08905898091569497, 52.69722175598818],
            [0.08905898091569497, 52.15527412683906],
            [0.9565339502005088, 52.15527412683906],
            [0.9565339502005088, 52.69722175598818],
            [0.08905898091569497, 52.69722175598818],
        ]
    ],
    "type": "Polygon",
}

items = rc.search(
    collections=["sentinel2_ard"],
    catalog_paths=["supported-datasets/ceda-stac-fastapi"],
    intersects=thetford_pnt,
    query=[
        "start_datetime>=2023-04-01",
        "end_datetime<=2023-06-30",
    ],
    limit=10,
)


For each item open `cog`, `cloud` and `valid` .tif assets and remove the cloud like this:


In [3]:
for item in items.get_limited()[0:1]:
    cloud_href = item.assets["cloud"].href
    valid_href = item.assets["valid_pixels"].href
    cog_href = item.assets["cog"].href
    print(item.id, cloud_href, valid_href, cog_href, sep="\n")
    valid = xr.open_dataset(valid_href, engine="rasterio", chunks="auto")
    cloud = xr.open_dataset(cloud_href, engine="rasterio", chunks="auto")
    cog = xr.open_dataset(cog_href, engine="rasterio", chunks="auto")
    print("==" * 10)



neodc.sentinel_ard.data.sentinel_2.2023.06.25.S2B_20230625_latn527lonw0007_T30UXD_ORB137_20230625114918_utm30n_osgb
https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/06/25/S2B_20230625_latn527lonw0007_T30UXD_ORB137_20230625114918_utm30n_osgb_clouds.tif
https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/06/25/S2B_20230625_latn527lonw0007_T30UXD_ORB137_20230625114918_utm30n_osgb_valid.tif
https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/06/25/S2B_20230625_latn527lonw0007_T30UXD_ORB137_20230625114918_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif


Warning 3: Cannot find gdalvrt.xsd (GDAL_DATA is not defined)


In [4]:
result = valid + cloud


In [6]:
# Set values greater than 1 to 0, others remain as 1
result = xr.where(result > 1, 0, 1)


In [7]:

# Multiply cog.tif by the result
final_result = cog * result
